In [17]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import network

from block import GENESIS_BLOCK
from helper import calculate_new_bits
from network import (
    NetworkEnvelope,
    VersionMessage,
)

### Exercise 1

Determine what this network message is:

`f9beb4d976657261636b000000000000000000005df6e0e2`

In [18]:
# Exercise 1
from network import NetworkEnvelope
from io import BytesIO

message_hex = 'f9beb4d976657261636b000000000000000000005df6e0e2'

# convert to binary
# see what the bytes 4 through 16 aregi
stream = BytesIO(bytes.fromhex(message_hex))
envelope = NetworkEnvelope.parse(stream)
print(envelope.command)
print(envelope.payload)

b'verack'
b''


### Exercise 2

Write the `parse` method for `NetworkEnvelope`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:NetworkEnvelopeTest:test_parse`

In [19]:
# Exercise 2

reload(network)
run(network.NetworkEnvelopeTest("test_parse"))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### Exercise 3

Write the `serialize` method for `NetworkEnvelope`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:NetworkEnvelopeTest:test_serialize`

In [20]:
# Exercise 3

reload(network)
run(network.NetworkEnvelopeTest("test_serialize"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 4

Write the `serialize` method for `VersionMessage`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:VersionMessageTest:test_serialize`

In [21]:
# Exercise 4

reload(network)
run(network.VersionMessageTest("test_serialize"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 5

Write the `handshake` method for `SimpleNode`

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:SimpleNodeTest:test_handshake`

In [22]:
# Exercise 5

reload(network)
run(network.SimpleNodeTest("test_handshake"))

E
ERROR: test_handshake (network.SimpleNodeTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/jose/dev/programmingbitcoin/code-ch10/network.py", line 363, in test_handshake
    node = SimpleNode('testnet.programmingbitcoin.com', testnet=True)
  File "/Users/jose/dev/programmingbitcoin/code-ch10/network.py", line 315, in __init__
    self.socket.connect((host, port))
OSError: [Errno 51] Network is unreachable

----------------------------------------------------------------------
Ran 1 test in 6.182s

FAILED (errors=1)


### Exercise 6

Write the `serialize` method for `GetHeadersMessage`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:GetHeadersMessageTest:test_serialize`

In [ ]:
# Exercise 6

reload(network)
run(network.GetHeadersMessageTest("test_serialize"))

In [ ]:
from io import BytesIO
from network import SimpleNode, GetHeadersMessage, HeadersMessage
from block import Block, GENESIS_BLOCK, LOWEST_BITS
from helper import calculate_new_bits
previous = Block.parse(BytesIO(GENESIS_BLOCK))
first_epoch_timestamp = previous.timestamp
expected_bits = LOWEST_BITS
count = 1
node = SimpleNode('mainnet.programmingbitcoin.com', testnet=False)
node.handshake()
for _ in range(19):
    getheaders = GetHeadersMessage(start_block=previous.hash())
    node.send(getheaders)
    headers = node.wait_for(HeadersMessage)
    for header in headers.blocks:
        if not header.check_pow():
            raise RuntimeError('bad PoW at block {}'.format(count))
        if header.prev_block != previous.hash():
            raise RuntimeError('discontinuous block at {}'.format(count))
        if count % 2016 == 0:
            time_diff = previous.timestamp - first_epoch_timestamp
            expected_bits = calculate_new_bits(previous.bits, time_diff)
            print(expected_bits.hex())
            first_epoch_timestamp = header.timestamp
        if header.bits != expected_bits:
            raise RuntimeError('bad bits at block {}'.format(count))
        previous = header
        count += 1